#IMPORT LIBRARY

In [ ]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import xgboost as xgb
from IPython.display import display
from pandas.plotting import scatter_matrix
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split, GridSearchCV, KFold, StratifiedKFold, cross_val_score
from sklearn.preprocessing import StandardScaler, MinMaxScaler, MaxAbsScaler, RobustScaler, QuantileTransformer, PowerTransformer
from sklearn.linear_model import LinearRegression,  Lasso, Ridge, ElasticNet
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor, AdaBoostRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.neural_network import MLPRegressor
from sklearn.svm import SVR
from sklearn.tree import DecisionTreeRegressor
from sklearn.base import BaseEstimator, RegressorMixin

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

# Data Overview

## 1. Load Dataframe

In [ ]:
root_dir = "/content/drive/MyDrive/dataset"
df_th = pd.read_csv(os.path.join(root_dir,'th-public.csv'))
df_tbtl = pd.read_csv(os.path.join(root_dir,'tbtl-public.csv'))
df_ck = pd.read_csv(os.path.join(root_dir,'ck-public.csv'))
df_qt = pd.read_csv(os.path.join(root_dir,'qt-public.csv'))
anno_df = pd.read_csv(os.path.join(root_dir,'annonimized.csv'))


## 2. Handle missing value

In [ ]:
df_th.dropna(subset=['TH'], inplace=True)
df_tbtl.dropna(subset=['TBTL'], inplace=True)
df_ck.dropna(subset=['CK'], inplace=True)
df_qt.dropna(subset=['diemqt'], inplace=True)


##3. Overview


In [ ]:
def overview(df):
    df.info()
    df.describe()
    df.dtypes
    print(df.nunique())
    df.shape

overview(anno_df)
overview(df_th)
overview(df_tbtl)
overview(df_ck)
overview(df_qt)


#

In [ ]:
display(anno_df.head(5))
display(df_tbtl.head(5))
display(df_qt.head(5))
display(df_th.head(5))
display(df_ck.head(5))

In [ ]:
rename_dict = {
    "concat('it001',`assignment_id`)": "assignment_id",
    "concat('it001',`problem_id`)": "problem_id",
    "concat('it001', username)": "username",
    "concat('it001',`language_id`)": "language_id"
}

anno_df.rename(columns=rename_dict, inplace=True)

df_tbtl.rename(columns={"username": "username"}, inplace=True)
df_th.rename(columns={"hash": "username"}, inplace=True)
df_qt.rename(columns={"hash": "username"}, inplace=True)
df_ck.rename(columns={"hash": "username"}, inplace=True)

In [ ]:
display(anno_df.head(5))
display(df_tbtl.head(5))
display(df_qt.head(5))
display(df_th.head(5))
display(df_ck.head(5))

#Feature Engineering

##1. Feature phản ánh độ chính xác


In [ ]:
#Sub = submission (lần nộp bài)
#Problem (bài tập nhỏ/ có nhiều problem trong 1 assignment)
#Assignment (bài tập lớn/ 1 bài tập lớn có nhiều problem)

#Tính số lần bài nộp được chấm đúng (xanh) của từng sinh viên:
right_sub = anno_df[(anno_df['is_final'] == 1) & (anno_df['pre_score'] == 10000)]
right_sub_count = right_sub.groupby('username').size().reset_index(name='right_sub_count')

#Tính số problem được chấm đúng của từng sinh viên:
right_pro_count = right_sub.groupby('username')['problem_id'].nunique().reset_index(name='right_pro_count')

#Tính số assignment được chấm đúng của từng sinh viên:
right_ass_count = right_sub.groupby('username')['assignment_id'].nunique().reset_index(name='right_ass_count')

#Đưa vào dataframe
anno_df = anno_df.merge(right_sub_count, on='username', how='left')
anno_df = anno_df.merge(right_pro_count, on='username', how='left')
anno_df = anno_df.merge(right_ass_count, on='username', how='left')




In [ ]:
#Xử lý giá trị thiếu
anno_df['right_sub_count'] = anno_df['right_sub_count'].fillna(0).astype(float)
anno_df['right_pro_count'] = anno_df['right_pro_count'].fillna(0).astype(float)
anno_df['right_ass_count'] = anno_df['right_ass_count'].fillna(0).astype(float)



In [ ]:
#Tính số lần nộp bài (submission count)
sub_count = anno_df.groupby('username').size().reset_index(name = 'sub_count')

#Tính số assignment mà sinh viên có làm (đúng và sai)
ass_count = anno_df.groupby('username')['assignment_id'].nunique().reset_index(name = 'ass_count')

#Tính số problem mà sinh viên có làm
pro_count = anno_df.groupby('username')['problem_id'].nunique().reset_index(name = 'pro_count')

#Thêm vào dataframe
anno_df = anno_df.merge(sub_count, on='username', how='left')
anno_df = anno_df.merge(ass_count, on='username', how='left')
anno_df = anno_df.merge(pro_count, on='username', how='left')

In [ ]:
right_sub = anno_df[(anno_df['is_final'] == 1) & (anno_df['pre_score'] == 10000)]
right_sub = right_sub.drop_duplicates(subset=['username', 'problem_id'], keep="first")

right_sub['score'] = (right_sub['pre_score'] / 10000) * (right_sub['coefficient'] / 100)
total_scores = right_sub.groupby('username')['score'].sum().reset_index(name='total_score')


In [ ]:
anno_df = anno_df.merge(total_scores, on='username', how='left')


In [ ]:
anno_df['total_score'] = anno_df['total_score'].fillna(0)

In [ ]:
status_dummies = pd.get_dummies(anno_df['status'], prefix='status_')
df_expanded = pd.concat([anno_df, status_dummies], axis=1)
df_grouped = df_expanded.groupby('username')[status_dummies.columns].sum().reset_index()
anno_df = anno_df.merge(df_grouped, on='username', how='left')


In [ ]:
del df_expanded, df_grouped

In [ ]:
#Tính tỉ lệ submit đúng so với tổng số lần submit
anno_df['right_sub_rate'] =anno_df["right_sub_count"] / anno_df["sub_count"] * 100
#Tính tỉ lệ problem đúng so với tổng problem tham gia
anno_df['right_pro_rate'] = anno_df["right_pro_count"] / anno_df["pro_count"] * 100
#Tính tỉ lệ problem đúng so với tổng số lần submit
anno_df["right_pro_sub_rate"] = anno_df["right_pro_count"] / anno_df["sub_count"] * 100


## 2. Feature phản ánh độ siêng năng

In [ ]:
#Chuyển đổi giá trị về dạng date time
anno_df['submission_date'] = pd.to_datetime(anno_df['created_at'], format='%m-%d %H:%M:%S', errors='coerce')
anno_df['submission_week'] = anno_df['submission_date'].dt.isocalendar().week


In [ ]:
# Tạo một cột ngày duy nhất từ 'submission_date'
anno_df['submission_day'] = anno_df['submission_date'].dt.date

# Tính số ngày làm bài
active_days = anno_df.groupby('username')['submission_day'].nunique().reset_index(name='active_days')

In [ ]:
# Tính tỉ lệ làm bài hàng ngày
sub_count = anno_df.groupby('username').size().reset_index(name='sub_count')

daily_submission_rate = pd.merge(sub_count, active_days, on='username', how='left')
daily_submission_rate['daily_submission_rate'] = daily_submission_rate['sub_count'] / daily_submission_rate['active_days']



In [ ]:
# 3. Số lần cải thiện điểm số
anno_df['score_improved'] = anno_df.groupby('username')['pre_score'].diff().apply(lambda x: 1 if x > 0 else 0)
improvements = anno_df.groupby('username')['score_improved'].sum().reset_index(name='improvement_count')
anno_df.drop(columns=['sub_count'], inplace=True)

In [ ]:
# Tổng hợp các đặc trưng về độ siêng năng
diligence_features = pd.merge(active_days, sub_count, on='username', how='left')
diligence_features = pd.merge(diligence_features, daily_submission_rate[['username', 'daily_submission_rate']], on='username', how='left')
diligence_features = pd.merge(diligence_features, improvements, on='username', how='left')

# Điền giá trị thiếu
diligence_features.fillna(0, inplace=True)

diligence_features

In [ ]:
anno_df = pd.merge(anno_df, diligence_features, on='username', how='left')

In [ ]:
anno_df

##3. Feature phản ảnh số lượng bài khó mà sinh viên giải

In [ ]:
assignments_per_problem = anno_df.groupby('problem_id')['assignment_id'].unique()
assignments_per_problem

In [ ]:
assignments_per_problem = anno_df.groupby('problem_id')['assignment_id'].unique()
students_per_assignment = anno_df.groupby('assignment_id')['username'].nunique()
students_per_problem = assignments_per_problem.apply(lambda x: students_per_assignment.loc[x].sum())
final_solutions = anno_df[(anno_df['is_final'] == 1) & (anno_df['pre_score'] == 10000)]
solved_counts = final_solutions.groupby('problem_id')['username'].nunique()
solved_ratio = (solved_counts / students_per_problem * 100).fillna(0)


In [ ]:
lv_df = pd.DataFrame({
    'problem_id': students_per_problem.index,
    'participated': students_per_problem.values,
    'solved': solved_counts.reindex(students_per_problem.index, fill_value=0).values,
    'ratio_solved/participated': solved_ratio.values
})
lv_df


In [ ]:
lv_df_sorted = lv_df.sort_values(by='ratio_solved/participated', ascending=False)

plt.figure(figsize=(8, 5))
plt.bar(range(len(lv_df_sorted)), lv_df_sorted['ratio_solved/participated'], color='red')
plt.xlabel('problem index')
plt.ylabel('Tỉ lệ')
plt.title('Tỉ lệ sinh viên giải được trên tổng số sinh viên tham gia mỗi problem')
plt.xticks(rotation=90)
plt.yticks(range(0, 101, 5))
plt.tight_layout()
plt.show()


In [ ]:
def classify_ratio(ratio):
    if ratio >= 60:
        return 'level_1'
    elif 35 <= ratio < 60:
        return 'level_2'
    elif 20 <= ratio < 35:
        return 'level_3'
    elif 10 <= ratio < 20:
        return 'level_4'
    else:
        return 'level_5'

# Tạo cột mới 'level' dựa trên các giá trị của cột 'ratio_solved/participated'
lv_df['level'] = lv_df['ratio_solved/participated'].apply(classify_ratio)
lv_df.to_csv('lv.csv', index=False)
display(lv_df)

In [ ]:
anno_df.info()

In [ ]:
anno_df = pd.merge(anno_df, lv_df[['problem_id', 'level']], on='problem_id', how='left')
ac_submissions = anno_df[(anno_df['is_final'] == 1) & (anno_df['pre_score'] == 10000)]
level_counts = ac_submissions.groupby(['username', 'level']).size().unstack(fill_value=0).reset_index()
anno_df = pd.merge(anno_df, level_counts, on='username', how='left')

level_columns = ['level_1', 'level_2', 'level_3', 'level_4', 'level_5']

for column in level_columns:
    anno_df[column] = anno_df[column].fillna(0).astype(int)


In [ ]:
anno_df.info()

In [ ]:
encoder = OneHotEncoder(sparse_output=False)
anno_df['problem_id'] = anno_df['problem_id'].str.strip()
# Thực hiện fit_transform trên cột 'assignment_id'
encoded_columns = encoder.fit_transform(anno_df[['problem_id']])
# Lấy tên các cột sau khi encoding và thêm tiền tố 'in__'
encoded_column_names = ['in__' + cat for cat in encoder.categories_[0]]
# Tạo DataFrame cho các cột đã được One-Hot Encoded
encoded_df = pd.DataFrame(encoded_columns, columns=encoded_column_names)
# Kết hợp DataFrame ban đầu với các cột mới
anno_df = pd.concat([anno_df, encoded_df], axis=1)

encoder3 = OneHotEncoder(sparse_output=False)
anno_df['assignment_id'] = anno_df['assignment_id'].str.strip()
# Thực hiện fit_transform trên cột 'assignment_id'
encoded3_columns = encoder3.fit_transform(anno_df[['assignment_id']])
# Lấy tên các cột sau khi encoding và thêm tiền tố 'in__'
encoded3_column_names = ['in__' + cat for cat in encoder3.categories_[0]]
# Tạo DataFrame cho các cột đã được One-Hot Encoded
encoded3_df = pd.DataFrame(encoded3_columns, columns=encoded3_column_names)
# Kết hợp DataFrame ban đầu với các cột mới
anno_df = pd.concat([anno_df, encoded3_df], axis=1)
display(anno_df)

In [ ]:
def prepare_train_predict_data(main_df, annotation_df):
    # Tạo bản sao của main_df
    train_data = main_df.copy()

    # Lấy danh sách username từ các DataFrame
    main_usernames = train_data['username']
    annotation_usernames = annotation_df['username']

    # Tìm các username có trong annotation_df nhưng không có trong main_df
    missing_usernames = annotation_usernames[~annotation_usernames.isin(main_usernames)]

    # Lọc các hàng trong annotation_df với các username thuộc missing_usernames
    filtered_data = annotation_df[annotation_df['username'].isin(missing_usernames)]

    # Lấy danh sách username duy nhất từ filtered_data
    unique_usernames = filtered_data['username'].unique()

    # Tạo DataFrame dự đoán
    predict_data = pd.DataFrame(unique_usernames, columns=['username'])

    # Cột dự đoán được lấy từ cột thứ hai trong main_df
    target_column = main_df.columns[1]

    # Đổi tên cột trong train_data theo từ điển rename_dict
    train_data.rename(columns=rename_dict, inplace=True)

    # Tóm tắt thông tin từ annotation_df theo username
    annotation_summary = annotation_df.groupby('username').first().reset_index()

    # Ghép dữ liệu từ annotation_summary vào train_data và predict_data
    train_data = pd.merge(train_data, annotation_summary, on='username', how='left')
    predict_data = pd.merge(predict_data, annotation_summary, on='username', how='left')

    # Các cột cần loại bỏ khỏi dữ liệu
    columns_to_remove = ['assignment_id', 'problem_id', 'is_final', 'status',
                         'pre_score', 'coefficient', 'language_id',
                         'created_at', 'updated_at', 'judgement', 'level',
                         'submission_date','submission_week','submission_day']

    # Loại bỏ các cột không cần thiết
    train_data.drop(columns=columns_to_remove, inplace=True)
    predict_data.drop(columns=columns_to_remove, inplace=True)

    # Loại bỏ các hàng có giá trị null trong cột mục tiêu
    train_data.dropna(subset=[target_column], inplace=True)

    return train_data, predict_data


train_df_TH, predict_df_TH = prepare_train_predict_data(df_th, anno_df)
train_df_QT, predict_df_QT = prepare_train_predict_data(df_qt, anno_df)
train_df_CK, predict_df_CK = prepare_train_predict_data(df_ck, anno_df)
train_df_TL, predict_df_TL = prepare_train_predict_data(df_tbtl, anno_df)

In [46]:
after_process_dir = "/content/drive/MyDrive/dataset/after_process"

#Save to CSV
train_df_TH.to_csv(os.path.join(after_process_dir,'train_df_TH.csv'), index=False)
train_df_QT.to_csv(os.path.join(after_process_dir,'train_df_QT.csv'), index=False)
train_df_CK.to_csv(os.path.join(after_process_dir,'train_df_CK.csv'), index=False)
train_df_TL.to_csv(os.path.join(after_process_dir,'train_df_TL.csv'), index=False)

predict_df_TH.to_csv(os.path.join(after_process_dir,'predict_df_TH.csv'), index=False)
predict_df_QT.to_csv(os.path.join(after_process_dir,'predict_df_QT.csv'), index=False)
predict_df_CK.to_csv(os.path.join(after_process_dir,'predict_df_CK.csv'), index=False)
predict_df_TL.to_csv(os.path.join(after_process_dir,'predict_df_TL.csv'), index=False)

# Training


##1. Spliting data

In [47]:
X_QT = train_df_QT.drop(columns=[train_df_QT.columns[1], 'username'])
y_QT = train_df_QT[train_df_QT.columns[1]]

X_TH = train_df_TH.drop(columns=[train_df_TH.columns[1], 'username'])
y_TH = train_df_TH[train_df_TH.columns[1]]

X_CK = train_df_CK.drop(columns=[train_df_CK.columns[1], 'username'])
y_CK = train_df_CK[train_df_CK.columns[1]]

X_TL = train_df_TL.drop(columns=[train_df_TL.columns[1], 'username'])
y_TL = train_df_TL[train_df_TL.columns[1]]


In [48]:
y_QT = y_QT.replace('\xa0', np.nan)  # Thay '\xa0' bằng NaN
y_QT = y_QT.fillna(0)  # Chuyển đổi kiểu dữ liệu về float
y_QT = y_QT.astype(float)  # Chuyển đổi kiểu dữ liệu về float

y_TH = y_TH.replace('\xa0', np.nan)  # Thay '\xa0' bằng NaN
y_TH = y_TH.fillna(0)  # Chuyển đổi kiểu dữ liệu về float
y_TH = y_TH.astype(float)  # Chuyển đổi kiểu dữ liệu về float

y_CK = y_CK.astype(float)
y_TL = y_TL.astype(float)

print(y_QT.unique())  # Hiển thị các giá trị duy nhất trong cột nhãn
print(y_TH.unique())  # Hiển thị các giá trị duy nhất trong cột nhãn
print(y_CK.unique())  # Hiển thị các giá trị duy nhất trong cột nhãn
print(y_TL.unique())  # Hiển thị các giá trị duy nhất trong cột nhãn

[ 7.5  7.   9.  10.   8.   9.5  6.5  8.5  6.   4.5  5.   1.5  5.5  0.
  1.   2.5  2.   3.5  4.   3. ]
[ 5.   8.5  7.  10.   6.   7.5  8.   5.5  9.   6.5  9.5  4.5  4.   2.5
  3.5  1.   1.5  2.   0.5  3.   0. ]
[ 5.5  6.5  4.5  5.   4.   8.   7.   6.   3.5  9.   7.5  8.5  3.  10.
  0.   9.5  2.   1.   2.5  0.5  1.5]
[7.24 8.11 7.3  8.63 8.2  7.5  6.42 7.23 7.34 8.59 7.41 8.8  8.01 8.6
 7.62 7.02 8.19 9.04 8.41 8.7  8.18 8.42 8.76 7.61 8.58 8.3  8.29 8.02
 8.32 6.6  8.86 7.89 9.13 7.28 7.97 7.56 8.71 8.5  7.93 7.81 6.77 7.64
 7.6  7.65 6.81 6.27 8.87 8.66 8.75 9.28 8.15 7.96 6.67 8.93 6.87 9.2
 8.34 7.83 9.   8.72 8.54 7.31 8.84 7.11 7.38 6.41 8.51 6.89 8.17 6.93
 9.07 8.53 7.17 7.7  5.89 8.43 7.21 8.65 8.94 7.82 6.61 6.43 2.52 8.1
 6.26 8.31 8.82 6.37 8.67 8.79 7.74 8.33 6.82 9.19 6.06 6.57 7.91 6.08
 8.99 7.78 7.66 7.19 8.44 8.69 6.46 7.25 7.48 8.52 7.08 8.06 7.54 8.
 7.59 7.22 8.4  7.33 7.49 6.68 7.8  8.07 8.38 8.12 7.09 7.44 6.09 7.51
 8.83 8.23 8.85 9.59 8.14 9.53 7.42 7.86 7.63 9.1

In [49]:
class WeightedVotingRegressor(BaseEstimator, RegressorMixin):
    def __init__(self, models, weights):
        self.models = models
        self.weights = weights

    def predict(self, X):
        predictions = np.array([model.predict(X) for model in self.models])
        weighted_predictions = np.average(predictions, axis=0, weights=self.weights)
        return weighted_predictions

def search_model(X, y, random_seed, models):
    best_models = []
    i = 0
    for name, model, params in models:
        pipeline = Pipeline([
            ('scaler', StandardScaler()),
            ('regressor', model)
        ])

        param_grid = {
            'scaler': [StandardScaler(), MinMaxScaler(), None]
        }
        param_grid.update(params)
        kf = KFold(n_splits=5, shuffle=True, random_state=random_seed)
        grid_search = GridSearchCV(pipeline, param_grid, cv=kf, scoring='r2', n_jobs=12, verbose = 1)
        grid_search.fit(X, y)

        cv_scores = cross_val_score(grid_search.best_estimator_, X, y, cv=kf, scoring='r2')
        mean_cv_score = np.mean(cv_scores)

        i += 1
        print(f"#{i} Model: {name}")
        print(f"Best parameters: {grid_search.best_params_}")
        print(f"Mean cross-validation score: {mean_cv_score}")
        best = grid_search.best_estimator_
        best_models.append(best)
        print('-' * 30)

    return best_models

In [ ]:
models = [
        #1
        ('linear', LinearRegression(), {
        }),
        #2
        ('ridge', Ridge(), {
            'regressor__alpha': [0.5, 1, 1.5, 2, 2.5, 3, 4, 5]
        }),
        #3
        ('lasso', Lasso(max_iter=5000), {
            'regressor__alpha':  [0.2, 0.4, 0.6, 0.8, 1]
        }),
        #4
        ('elastic_net', ElasticNet(max_iter=5000), {
            'regressor__alpha': [0.1, 0.3, 0.5, 1, 1.5, 2, 2.5, 3],
            'regressor__l1_ratio': [0.2, 0.4, 0.6, 0.8, 1]
        }),
        #5
        ('svr', SVR(), {
            'regressor__C': [1.0, 2.0, 3.0, 4.0, 5.0, 6.0 ,7.0],
            'regressor__kernel': ['rbf'],
            'regressor__epsilon': [0.2, 0.4, 0.6, 0.8, 1]
        }),
        # #6
        ('random_forest', RandomForestRegressor(random_state=42
            'regressor__n_estimators': [50, 70, 100],
            'regressor__max_depth': [None, 3, 5, 7, 10]
        }),
        #7
        ('knn_regressor', KNeighborsRegressor(), {
            'regressor__n_neighbors': [3, 5, 10],
            'regressor__weights': ['uniform', 'distance']
        }),
        #8
        ('adaboost', AdaBoostRegressor(random_state=42), {
            'regressor__n_estimators': [10, 30, 50, 100, 200, 300],
            'regressor__learning_rate': [0.01, 0.03, 0.1]
        }),
    ]

In [50]:
best_models_QT = search_model(X_QT, y_QT, random_seed=42, models=models)


Fitting 5 folds for each of 3 candidates, totalling 15 fits
#1 Model: linear
Best parameters: {'scaler': None}
Mean cross-validation score: 0.2659576552510249
------------------------------
Fitting 5 folds for each of 24 candidates, totalling 120 fits
#2 Model: ridge
Best parameters: {'regressor__alpha': 2.5, 'scaler': MinMaxScaler()}
Mean cross-validation score: 0.3281923489812457
------------------------------
Fitting 5 folds for each of 15 candidates, totalling 75 fits
#3 Model: lasso
Best parameters: {'regressor__alpha': 0.2, 'scaler': StandardScaler()}
Mean cross-validation score: 0.18037724151464468
------------------------------
Fitting 5 folds for each of 120 candidates, totalling 600 fits
#4 Model: elastic_net
Best parameters: {'regressor__alpha': 0.1, 'regressor__l1_ratio': 0.4, 'scaler': StandardScaler()}
Mean cross-validation score: 0.3221879893848848
------------------------------
Fitting 5 folds for each of 105 candidates, totalling 525 fits
#5 Model: svr
Best parameters:

In [ ]:
best_models_TH = search_model(X_TH, y_TH, random_seed=42, models=models)


Fitting 5 folds for each of 3 candidates, totalling 15 fits
#1 Model: linear
Best parameters: {'scaler': None}
Mean cross-validation score: 0.39124507016540383
------------------------------
Fitting 5 folds for each of 24 candidates, totalling 120 fits
#2 Model: ridge
Best parameters: {'regressor__alpha': 1, 'scaler': MinMaxScaler()}
Mean cross-validation score: 0.4210451853910106
------------------------------
Fitting 5 folds for each of 15 candidates, totalling 75 fits
#3 Model: lasso
Best parameters: {'regressor__alpha': 0.2, 'scaler': None}
Mean cross-validation score: 0.2844713542853632
------------------------------
Fitting 5 folds for each of 120 candidates, totalling 600 fits
#4 Model: elastic_net
Best parameters: {'regressor__alpha': 0.1, 'regressor__l1_ratio': 0.2, 'scaler': StandardScaler()}
Mean cross-validation score: 0.404027889418204
------------------------------
Fitting 5 folds for each of 105 candidates, totalling 525 fits
#5 Model: svr
Best parameters: {'regressor__C

In [51]:
best_models_CK = search_model(X_CK, y_CK, random_seed=42, models=models)


Fitting 5 folds for each of 3 candidates, totalling 15 fits


KeyboardInterrupt: 

In [44]:
best_models_TL = search_model(X_TL, y_TL, random_seed=42, models=models)

Fitting 5 folds for each of 3 candidates, totalling 15 fits
#1 Model: linear
Best parameters: {'scaler': StandardScaler()}
Mean cross-validation score: 0.11608098348798779
------------------------------
Fitting 5 folds for each of 24 candidates, totalling 120 fits
#2 Model: ridge
Best parameters: {'regressor__alpha': 5, 'scaler': None}
Mean cross-validation score: 0.19482610322520366
------------------------------
Fitting 5 folds for each of 15 candidates, totalling 75 fits
#3 Model: lasso
Best parameters: {'regressor__alpha': 0.2, 'scaler': None}
Mean cross-validation score: 0.10776984233018791
------------------------------
Fitting 5 folds for each of 120 candidates, totalling 600 fits
#4 Model: elastic_net
Best parameters: {'regressor__alpha': 0.1, 'regressor__l1_ratio': 0.2, 'scaler': StandardScaler()}
Mean cross-validation score: 0.14924283363990112
------------------------------
Fitting 5 folds for each of 105 candidates, totalling 525 fits
#5 Model: svr
Best parameters: {'regres

In [45]:
search_model(train_df_QT.drop(columns=[train_df_QT.columns[1], 'mssv_h']), train_df_QT[train_df_QT.columns[1]], 42, models)

KeyError: "['diemqt' 'mssv_h'] not found in axis"

In [53]:
model_QT1 = Pipeline(steps=[('scaler', MinMaxScaler()),
                            ('regressor', SVR(C=5.0, epsilon=0.4))])
model_QT1.fit(X_QT, y_QT)

model_QT2 = Pipeline(steps=[('scaler', StandardScaler()),
                 ('regressor', ElasticNet(alpha=0.1, l1_ratio=0.4, max_iter=100000))])
model_QT2.fit(X_QT, y_QT)

list_model_QT = [model_QT1, model_QT2]
list_w_QT = [0.5, 0.5]
voting_QT = WeightedVotingRegressor(models=list_model_QT, weights=list_w_QT)

Tìm mô hình và tham số cho dự đoán điểm thực hành

In [57]:
search_model(train_df_TH.drop(columns=[train_df_TH.columns[1], 'username']), train_df_TH[train_df_TH.columns[1]], 42, models)

KeyError: "['TH' 'mssv_h'] not found in axis"

In [58]:
model_TH1 = Pipeline(steps=[('scaler', MinMaxScaler()), ('regressor', Ridge(alpha=2.5))])
model_TH1.fit(X_TH, y_TH)

model_TH2 = Pipeline(steps=[('scaler', MinMaxScaler()),
                 ('regressor', SVR(C=7.0, epsilon=0.4))])
model_TH2.fit(X_TH, y_TH)

list_model_TH = [model_TH1, model_TH2]
list_w_TH = [0.5, 0.5]
voting_TH = WeightedVotingRegressor(models=list_model_TH, weights=list_w_TH)

Tìm mô hình và tham số cho dự đoán điểm cuối kỳ

In [ ]:
search_model(train_df_CK.drop(columns=[train_df_CK.columns[1], 'mssv_h']), train_df_CK[train_df_CK.columns[1]], 42, models)

In [55]:
model_CK1 = Pipeline(steps=[('scaler', MinMaxScaler()), ('regressor', Ridge(alpha=4))])
model_CK1.fit(X_CK, y_CK)

model_CK2 = Pipeline(steps=[('scaler', MinMaxScaler()),
                 ('regressor', SVR(C=3.0, epsilon=0.6))])
model_CK2.fit(X_CK, y_CK)

list_model_CK = [model_CK1, model_CK2]
list_w_CK = [0.5, 0.5]
voting_CK = WeightedVotingRegressor(models=list_model_CK, weights=list_w_CK)

Tìm mô hình và tham số cho dự đoán điểm trung bình tích lũy

In [ ]:
search_model(train_df_TL.drop(columns=[train_df_TL.columns[1], 'mssv_h']), train_df_TL[train_df_TL.columns[1]], 42, models)

In [54]:
model_TL1 = Pipeline(steps=[('scaler', MinMaxScaler()), ('regressor', Ridge(alpha=5))])
model_TL1.fit(X_TL, y_TL)

model_TL2 = Pipeline(steps=[('scaler', MinMaxScaler()),
                 ('regressor', SVR(C=2.0, epsilon=0.4))])
model_TL2.fit(X_TL, y_TL)

list_model_TL = [model_TL1, model_TL2]
list_w_TL = [0.5, 0.5]
voting_TL = WeightedVotingRegressor(models=list_model_TL, weights=list_w_TL)

In [59]:
def predict_submit(model, df, desc, predict_col):
    tdf = df.copy()
    predicted_target = model.predict(tdf.drop(columns=['username']))
    tdf['target'] = predicted_target
    tdf['target'] = tdf['target'].apply(lambda x: min(max(x, 0), 10))
    output_df = tdf[['username', 'target']]
    output_df.to_csv(f'out_{predict_col}_{desc}.csv', index=False, header=False)

predict_submit(voting_QT, predict_df_QT, 'final', 'QT')

predict_submit(voting_TH, predict_df_QT, 'final', 'TH')

predict_submit(voting_CK, predict_df_CK, 'final', 'CK')

predict_submit(voting_TL, predict_df_TL, 'final', 'TL')